# Datagrokr Internship Assignement

Dataset : **Stack Overflow 2019 Developer Survey Results**.

### Importing libraries 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import collections as cl
from datetime import datetime

- numpy and pandas to work with the Dataset.<br>
- seaborn and matplotlib for Visualization of the data.<br>


### Importing Dataset

In [ ]:
data = pd.read_csv('/kaggle/input/stack-overflow-developer-survey-results-2019/survey_results_public.csv')

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
desc = pd.read_csv('../input/stack-overflow-developer-survey-results-2019/survey_results_schema.csv')
desc.values

In [ ]:
data.drop(columns={'Respondent','Employment','Student','EdLevel','UndergradMajor','EduOther','OrgSize','DevType','YearsCode','YearsCodePro','MgrIdiot','MgrMoney','MgrWant','JobSeek','LastHireDate','LastInt','FizzBuzz','JobFactors','CurrencySymbol','CurrencyDesc','CompTotal','CompFreq','WorkPlan','WorkChallenge','WorkRemote','ImpSyn','CodeRev','CodeRevHrs','UnitTests','PurchaseHow','PurchaseWhat','MiscTechWorkedWith','MiscTechDesireNextYear','DevEnviron','Containers','BlockchainOrg','BlockchainIs','BetterLife','ITperson','OffOn','Extraversion','ScreenName','SOVisit1st','SOVisitFreq','SOVisitTo','SOFindAnswer','SOTimeSaved','SOHowMuchTime','SOAccount','SOPartFreq','SOJobs','EntTeams','SOComm','WelcomeChange','OpenSource','SONewContent','Trans','Sexuality','Ethnicity','Dependents','SurveyLength','SurveyEase'},inplace = True)

#### Basic understanding(statistical) of the dataset 

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

## Data Preprocessing

Type Convertion
- MainBranch to Category type
- Age to Integer type as age is always a whole number.
- Gender to Category type as 'Male' or 'Female'.

Quantity of incompleteness which needs to be sorted else it will hamper the analysis
- MainBranch Will remove all the empty data as Non-Developer.
- Country We don't have other option as to remove the entire row without Country column.
- WorkWeekHrs We will replace the empty entry with mean of that column.
- Gender We will take the empty entry as Transgender as they sometimes do not state their Gender.

### Cleaning The Data.

So now we know the datatypes of the columns and hence now we will be converting them to their required data tyes. 

#### Let's Start with MainBranch to Category type and from MainBranch we will remove all the empty data as Student.

In [ ]:
data['MainBranch'].value_counts()

#### Next we will convert CareerSat to category type.

In [ ]:
data['CareerSat'].value_counts()

In [ ]:
data['CareerSat'] = data['CareerSat'].astype('category')
print(data['CareerSat'])

#### Next we will convert Age to Integer type as age is always a whole number.

In [ ]:
Age = []
count = 0
for i in data['Age']:
    try:
        Age.append(int(i))
    except:
        Age.append(i)
        count += 1
data['Age'] = Age

In [ ]:
data['Gender'].value_counts()

#### For Country, We don't have other option as to remove the entire row without Country column.

In [ ]:
Country = data['Country'].value_counts().index
filters = []
for i in data['Country']:
    filters.append(i in Country)
    
data = data[filters]

In [ ]:
data.info()

In [ ]:
data.shape

### 1.Find the average age of developers when they wrote their first line of code.

- Replacing String values "Younger than 5 years" and "Older than 85" with numeric values 5 and 85 for numerical computability

In [ ]:
data["Age1stCode"].replace({"Younger than 5 years": "5","Older than 85": "85"}, inplace=True)
dev = data[data.MainBranch.str.contains("developer", na=False)]
xyz = pd.to_numeric(dev['Age1stCode'])
xyz.mean()


### 2. Deduce the percentage of developers who know python in each country.

Checking whether a developer --> finding language worked with (Python) --> grouping with countries

In [ ]:
dev = data[data.MainBranch.str.contains("developer", na=False)]
pythonistas = dev[dev.LanguageWorkedWith.str.contains("Python", na=False)]
xyz = pythonistas['Country'].value_counts()
print(xyz)

### 3. Generate a report for the average salary of developer based on continent.

- Grouping all the countries to its respective continents 
- Asia, Africa, Europe, NA(North America), SA(South America), Other(Australia and island groups & Antartica)

In [ ]:
Asia = ['Afghanistan','Armenia','Azerbaijan','Bahrain','Bangladesh','Bhutan','Brunei','Cambodia','China','East Timor','Georgia','India','Indonesia','Iran','Iraq','Israel','Japan','Jordan','Kazakhstan','Kuwait',
'Kyrgyzstan','Laos','Lebanon','Malaysia','Maldives','Mongolia','Myanmar','Nepal','North Korea','Oman','Pakistan','Philippines','Qatar','Russia','Saudi Arabia','Singapore','South Korea','Sri Lanka',
'Syria','Tajikistan','Thailand','Turkey','Turkmenistan','Taiwan','United Arab Emirates','UAE','Uzbekistan','Vietnam','Yemen']

Europe = ['Albania','Andorra','Austria' ,'Belarus' ,'Belgium', 'Bosnia and Herzegovina' ,'Bulgaria','Croatia','Cyprus','Czech Republic','Denmark', 'Estonia' ,'Finland' ,'France' ,'Georgia' ,'Germany', 'Greece',
'Hungary','Iceland','Republic of Ireland','Italy' ,'Kazakhstan','Kosovo','Latvia','Liechtenstein','Lithuania','Luxembourg' ,'North Macedonia','Malta' ,'Moldova','Monaco', 'Montenegro','Netherlands','Norway', 
'Poland','Portugal','Romania', 'San Marino','Serbia', 'Slovakia','Slovenia','Spain','Sweden','Switzerland', 'Turkey','Ukraine', 'United Kingdom','UK','Vatican City' ]



Africa = ['Algeria','Angola','Benin','Botswana','Burkina Faso','Burundi', 'Cameroon', 'Cape Verde','Central African Republic','Chad', 'Comoros','Republic of the Congo','Democratic Republic of the Congo', 
'Ivory Coast','Djibouti','Egypt','Equatorial Guinea','Eritrea','Ethiopia','Gabon', 'Gambia','Ghana','Guinea','Guinea-Bissau','Kenya','Lesotho','Liberia','Libya' ,'Madagascar','Malawi','Mali' ,'Mauritania','Mauritius','Morocco' ,'Mozambique','Namibia','Niger' ,'Nigeria' ,'Rwanda','São Tomé and Príncipe','Senegal' ,'Seychelles' ,'Sierra Leone','Somalia','South Africa' ,'South Sudan' ,'Sudan','Swaziland','Tanzania','Togo' ,'Tunisia' ,'Uganda','Western Sahara','Zambia','Zimbabwe']

NA = ['Antigua and Barbuda','Anguilla','Aruba','Bahamas','Barbados','Belize','Bermuda','Bonaire','British Virgin Islands','Canada','Cayman Islands','Clipperton Island','Costa Rica' ,'Cuba','Curaçao', 'Dominica','Dominican Republic','El Salvador','Greenland','Grenada','Guadeloupe','Guatemala','Haiti','Honduras','Jamaica' ,'Martinique','Mexico' ,'Montserrat','Navassa Island','Nicaragua' ,'Panama', 
'Puerto Rico','Saba','Saint Barthelemy','Saint Kitts and Nevis','Saint Lucia','Saint Martin','Saint Pierre and Miquelon','Saint Vincent and the Grenadines','Sint Eustatius' ,'Sint Maarten','Trinidad and Tobago','Turks and Caicos','United States of America','US','USA','United States','US Virgin Islands' ]

SA = ['Argentina','Bolivia','Brazil','Chile','Colombia','Ecuador', 'Falkland Islands','French Guiana', 'Guyana','Paraguay','Peru','South Georgia and the South Sandwich Islands','Suriname','Uruguay','Venezuela' ]

other = ['Antartica','Australia','Federated States of Micronesia','Fiji','Kiribati','Marshall Islands','Nauru','New Zealand','Palau', 'Papua New Guinea','Samoa','Solomon Islands','Tonga', 'Tuvalu','Vanuatu']

def GetConti(Country):
    if Country in Asia:
        return "Asia"
    elif Country in Europe:
        return "Europe"
    elif Country in Africa:
        return "Africa"
    elif Country in NA:
        return "NA"
    elif Country in SA:
        return "SA"
    else:
        return "Other"

#df = pd.DataFrame({"Country":[]})
#df = pd.read_csv('/kaggle/input/stack-overflow-developer-survey-results-2019/survey_results_public.csv')
data['Continent'] = data['Country'].apply(lambda x: GetConti(x))
#print(data['Continent'])

data['Continent'].value_counts()

In [ ]:
num = data['Continent'].value_counts()
total = data.groupby('Continent')['ConvertedComp'].sum()
data_plot1 = (total/num).sort_values(ascending=False)
data_plot = data_plot1.reset_index()
data_plot.rename(columns={0:'Average income in USD','index':'Continent'},inplace=True)
sns.barplot(y='Continent',x='Average income in USD',data = data_plot)


### 4. What is the distribution of people who code as a hobby based on gender and continent?

- Forming a group/category by replacing values for easy computation

In [ ]:
total = data.groupby('Continent')['Gender']
total.value_counts()
xyz = data.groupby('Hobbyist')['Gender']
xyz.value_counts()


### 5. Generate the report for job and career satisfaction of developer based on their gender and continent?

- Categorising gender (Male, Female, Other)

In [ ]:
data["Gender"].replace({"Man": "Male","Man;Non-binary, genderqueer, or gender non-conforming": "Male","Woman": "Female","Woman;Non-binary, genderqueer, or gender non-conforming": "Female","Woman;Man;Non-binary, genderqueer, or gender non-conforming": "Other"," Non-binary, genderqueer, or gender non-conforming" : "Other","Woman;Man" : "Other","Non-binary, genderqueer, or gender non-conforming" : "Other"}, inplace=True)
data['Gender'].value_counts()

### Career Satisfaction

- We can't group career satisfaction and job satisfaction because there are few rows where results converse and grouping them will contadict the inference
- Grouping satisfied and dissatisfied will reduce the group variable otherwise there will 3 categories( satisfied, neutral and disatisfied)

In [ ]:
dev = data[data.MainBranch.str.contains("developer", na=False)]
pythonistas = dev[dev.CareerSat.str.contains("satisfied", na=False)]
#xyz = pythonistas['Country'].value_counts()
xyz = pythonistas.groupby('Gender')['Continent']
xyz.value_counts()

- Career Dissatisfaction

In [ ]:
dev = data[data.MainBranch.str.contains("developer", na=False)]
pythonistas = dev[dev.CareerSat.str.contains("dissatisfied", na=False)]
#xyz = pythonistas['Country'].value_counts()
xyz = pythonistas.groupby('Gender')['Continent']
xyz.value_counts()

- Job Satisfaction

In [ ]:
dev = data[data.MainBranch.str.contains("developer", na=False)]
pythonistas = dev[dev.JobSat.str.contains("satisfied", na=False)]
#xyz = pythonistas['Country'].value_counts()
xyz = pythonistas.groupby('Gender')['Continent']
xyz.value_counts()

- Job Dissatisfied

In [ ]:
dev = data[data.MainBranch.str.contains("developer", na=False)]
pythonistas = dev[dev.JobSat.str.contains("dissatisfied", na=False)]
#xyz = pythonistas['Country'].value_counts()
xyz = pythonistas.groupby('Gender')['Continent']
xyz.value_counts()

### 6. For problems f and g, gender should have three categories man, woman and others. If a record have any other value, consider it as Others.

In [ ]:
data['Gender'].value_counts()

### 7. Based on this survey, what will be the most desired programming language for the year 2020?


#### Let's Start with MainBranch to Category type and from MainBranch we will remove all the empty data as Student.

In [ ]:
MainBranch = []
for i in data['MainBranch']:
    if i == 'I am a developer by profession':
        MainBranch.append('Developer')
    elif i == 'I am a student who is learning to code':
        MainBranch.append('Student')
    elif i == 'I am not primarily a developer, but I write code sometimes as part of my work':
        MainBranch.append('Semi_Developer')
    elif i == 'I code primarily as a hobby':
        MainBranch.append('Hobby') 
    elif i == 'I used to be a developer by profession, but no longer am':
        MainBranch.append('Ex_Developer')
    else:
        MainBranch.append('Student')
data['MainBranch'] = MainBranch
data['MainBranch'] = data['MainBranch'].astype('category',inplace=True)

- Forming a word cloud by spliting the data, using semicolon as terminate value

In [ ]:
data_dev = data[data['MainBranch']=='Developer']

In [ ]:
def generate_word_column_for_the_column_of(column):
    column_name = column
    os_now_all_word = ''
    for i in data_dev[column_name]:
        try:
            a=i.split(';')
            for j in a:
                os_now_all_word+=' '+ j
        except:
            a=5

    cloud = WordCloud(background_color="white",max_font_size=250,width=960, height=1080).generate(os_now_all_word)

    cloud.to_file(column_name + '.png')
    plt.imshow(cloud)

In [ ]:
generate_word_column_for_the_column_of('LanguageDesireNextYear')

## INFERENCE

#### 1.Find the average age of developers when they wrote their first line of code.
ANS : 15.393174052214313

#### 2.Deduce the percentage of developers who know python in each country.
ANS : Top 5 is represented here rest can be infered from que 2.
*       United States     8541
*       India             2179
*       United Kingdom    1955
*       Germany           1943
*       Canada            1258 ...
      
#### 3.Generate a report for the average salary of developer based on continent.
ANS : Can be infered from the graph
*       NA     ~ 160k
*       Other  ~ 80k
*       EU     ~ 70k
*       SA     ~ 30k
*       Asia   ~ 20k
*       Africa ~ 19k
      
#### 4.What is the distribution of people who code as a hobby based on gender and continent?
ANS :  
Hobbyist  Gender                                                     
* No     
          Man                                                            14300
          Woman                                                           2318
          Non-binary, genderqueer, or gender non-conforming                119
          Woman;Non-binary, genderqueer, or gender non-conforming           32
          Man;Non-binary, genderqueer, or gender non-conforming             28
          Woman;Man                                                         22
          Woman;Man;Non-binary, genderqueer, or gender non-conforming       15
* Yes     
          Man                                                            63619
          Woman                                                           4026
          Non-binary, genderqueer, or gender non-conforming                478
          Man;Non-binary, genderqueer, or gender non-conforming            153
          Woman;Non-binary, genderqueer, or gender non-conforming          131
          Woman;Man                                                        110
          Woman;Man;Non-binary, genderqueer, or gender non-conforming       55
      
### 5. Generate the report for job and career satisfaction of developer based on their gender and continent?

- Career Satisfaction and Dissatisfication
* Gender  Continent
* Female                                  
        NA            2105          NA            395
        Europe        1654          Europe        418
        Asia           822          Asia          303
        Other          272          Other          69
        SA             157          SA             47
        Africa         120          Africa         38
* Male   
        Europe       25399          Europe       6225
        NA           18657          NA           3371
        Asia         12099          Asia         4273
        Other         4168          Other        1140
        SA            2741          SA            862
        Africa        1611          Africa        504
* Other  
        NA             237          NA             69
        Europe         164          Europe         48
        Asia           109          Asia           50
        Other           35          Other          15
        Africa          12          Africa          8
        SA               9          SA              5
        
        
- Job Statisfaction and Dissatisfication
* Gender  Continent
* Female                                  
        NA            2037          NA            517
        Europe        1619          Europe        522
        Asia           785          Asia          338
        Other          266          Other          81
        SA             149          SA             54
        Africa         114          Africa         36
* Male   
        Europe       25023          Europe       8172
        NA           18275          NA           5127
        Asia         11725          Asia         5178
        Other         4079          Other        1475
        SA            2664          SA            1108
        Africa        1475          Africa        617
* Other  
        NA             229          NA             75
        Europe         158          Europe         56
        Asia           104          Asia           52
        Other           32          Other          14
        Africa          11          Africa          6
        SA               8          SA              5
        
### 6. For problems f and g, gender should have three categories man, woman and others. If a record have any other value, consider it as Others.

* Male      78100
* Female     6507
* Other       799

### 7. Based on this survey, what will be the most desired programming language for the year 2020?

1. HTML
2. CSS
3. Javascript
4. Shell
5. Bash, Powershell...